In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import numba
from fracdiff.frac_weights import frac_weights

# Load Data

In [3]:
with np.load('data/demo1.npz') as data:
    x = data['px'][:, 0]

In [4]:
w = frac_weights(0.345, 100)
w = np.array(w)

# Variants

In [5]:
def apply_weights_1(x: np.ndarray, w: list) -> np.ndarray:
    m = len(w)
    z = w[0] * x
    z[:(m - 1)] = np.nan
    for k in range(1, m):
        z[k:] += w[k] * x[:-k]
    return z

In [6]:
@numba.jit(nopython=True)
def apply_weights_2(x: np.ndarray, w: np.array) -> np.ndarray:
    m = w.shape[0]
    z = w[0] * x
    z[:(m - 1)] = np.nan
    for k in range(1, m):
        z[k:] += w[k] * x[:-k]
    return z

# Pre-Run

In [7]:
%time _ = apply_weights_1(x, w)
%time _ = apply_weights_2(x, w)

CPU times: user 2.95 ms, sys: 2.15 ms, total: 5.1 ms
Wall time: 11.1 ms
CPU times: user 607 ms, sys: 8.83 ms, total: 616 ms
Wall time: 627 ms


# Speed

In [8]:
%timeit _ = apply_weights_1(x, w)
%timeit _ = apply_weights_2(x, w)

874 µs ± 8.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
332 µs ± 5.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
